In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px

import plotly.graph_objects as go # or plotly.express as px
fig = go.Figure() # or any Plotly Express function e.g. px.bar(...)

import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps
import base64

from PythonScript import CRUD



###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "AACmongo"

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(CRUD.readMany(username, password, {}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GSL.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Christian Kesler'))),
    html.Hr(),
    
    html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())), href="https://www.snhu.edu/"),
    html.Hr(),
    html.Div(className='row',
        style={'display' : 'flex'},
        children=[
            html.Button(id='submit-button-one', n_clicks=0, children='Water'),
            html.Button(id='submit-button-two', n_clicks=0, children='Mountain or Wilderness'),
            html.Button(id='submit-button-three', n_clicks=0, children='Disaster or Individual Tracking'),
            html.Button(id='submit-button-four', n_clicks=0, children='Reset')
        ]
    ),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            
            )
        ])
])



#############################################
# Interaction Between Components / Controller
#############################################


# This callback highlights a selected column
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback creates the pie chart based on the table data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)    
    fig = px.pie(dff, names='breed')
    pieChart = [
        dcc.Graph(figure=fig)
    ]
    return pieChart
    
    
# This callback creates the geolocation chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    marker = dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
        dl.Tooltip(dff.iloc[0,4]),
        dl.Popup([
            html.H1("Animal Name"),
            html.P(dff.iloc[0,9])
        ])
    ])
    geoMap = [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            marker
        ])
    ]
    return geoMap
    
    
# This callback applies filters when the button is clicked
@app.callback(
    Output('datatable-id', "data"),
    [Input('submit-button-one', 'n_clicks'),
     Input('submit-button-two','n_clicks'),
     Input('submit-button-three', 'n_clicks'),
     Input('submit-button-four', 'n_clicks'),
    ])
def on_clicks(bt1,bt2,bt3,bt4):
    #start case
    if (int(bt1) == 0 and int(bt2) == 0 and int(bt3) == 0 and int(bt4) == 0):
        df = pd.DataFrame.from_records(CRUD.readMany(username, password, {}))
    elif (int(bt1) > int(bt2) and int(bt1) > int(bt3) and int(bt1) > int(bt4)):
        df = pd.DataFrame.from_records(CRUD.readMany(username, password, {
            "breed" : { '$in': ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, 
            "sex_upon_outcome" : "Intact Female",
            "age_upon_outcome" : { '$in': ["6 months", "7 months", "8 months", "9 months", "10 months", "11 months", "1 year", "2 years", "3 years"]}
        }))
    elif (int(bt2) > int(bt1) and int(bt2) > int(bt3) and int(bt2) > int(bt4)):
        df = pd.DataFrame.from_records(CRUD.readMany(username, password, {
            "breed" : { '$in': ["German Shepherd", "Alaskan", "Malamute", "Old English", "Sheepdog", "Siberian Husky", "Rottweiler"]}, 
            "sex_upon_outcome" : "Intact Male",
            "age_upon_outcome" : { '$in': ["6 months", "7 months", "8 months", "9 months", "10 months", "11 months", "1 year", "2 years", "3 years"]}
        }))
    elif (int(bt3) > int(bt2) and int(bt3) > int(bt1) and int(bt3) > int(bt4)):
        df = pd.DataFrame.from_records(CRUD.readMany(username, password, {
            "breed" : { '$in': ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, 
            "sex_upon_outcome" : "Intact Male",
            "age_upon_outcome" : { '$in': ["5 months", "6 months", "7 months", "8 months", "9 months", "10 months", "11 months", "1 year", "2 years", "3 years", "4 years", "5 years"]}
        }))
    elif (int(bt4) > int(bt2) and int(bt4) > int(bt3) and int(bt4) > int(bt1)):
        df = pd.DataFrame.from_records(CRUD.readMany(username, password, {}))
    return df.to_dict('records')

app

Attempting to Read. . . 
Reading successful!
